In [175]:
# Useful for defining quantities
from astropy import units as u
from astropy.coordinates import cartesian_to_spherical

# Earth focused modules, ISS example orbit and time span generator
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.twobody import Orbit
from poliastro.spacecraft import Spacecraft
from poliastro.examples import iss
from poliastro.util import time_range
from poliastro.bodies import Earth

In [130]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

In [25]:
radius_E = 6400

In [107]:
store = lambda: None

In [237]:
@interact(h=FloatSlider(min=7000, max=19000,value=11500, step=100, continuous_update=False),
          i=FloatSlider(min=0, max=90, step=1, continuous_update=False),
          duration=IntSlider(min=20, max=2000, step=10, continuous_update=False),
          max_day_bn_rep=IntSlider(value=2,min=1, max=30, step=1, continuous_update=False))
def groundtract(h, i, duration,max_day_bn_rep):
    # Build spacecraft instance
    a = (radius_E+h) * u.km
    ecc = 0 * u.one
    inc = i * u.deg
    raan = 0 * u.deg
    argp = 0 * u.deg
    nu = 0 * u.deg
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
    print("Original period: ", orb.period.to(u.h))
    #adjust to integer orbits in a day
    T_E =  Earth.rotational_period.to(u.s)
    ratio =T_E/orb.period 
    ratio = np.floor(max_day_bn_rep*ratio)/float(max_day_bn_rep)*u.one
    new_period = T_E/ratio
    a = (Earth.k*(new_period/2/np.pi)**2)**(1./3)
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

    print("Adjusted period: ", orb.period.to(u.h), "Adjusted altitude: ", (a - radius_E*u.km).to(u.km))
    sat = EarthSatellite(orb, None)
    t_span = time_range(orb.epoch, periods=5*duration, end= orb.epoch + duration*u.h)
    gp = GroundtrackPlotter()
    gp.update_layout(title="Groundtrack")

    # Plot previously defined EarthSatellite object
    gp.plot(
        sat,
        t_span,
        label="Sat",
        color = "red",
    )
    store.gp_groundtract = gp
    store.sat = sat
    store.t_span = t_span
    store.t_end = duration*u.h
    gp.fig.show()

interactive(children=(FloatSlider(value=11500.0, continuous_update=False, description='h', max=19000.0, min=70…

In [112]:
store.gp_groundtract.update_geos(projection_type="orthographic")


In [211]:
store.sat.orbit.sample?

In [186]:
store.sat.orbit.sample(10).xyz

<Quantity [[ 1.82907532e+04,  1.40115299e+04,  3.17615596e+03,
            -9.14537661e+03, -1.71876858e+04, -1.71876858e+04,
            -9.14537661e+03,  3.17615596e+03,  1.40115299e+04,
             1.82907532e+04],
           [ 0.00000000e+00,  1.09761700e+04,  1.68164681e+04,
             1.47881538e+04,  5.84029807e+03, -5.84029807e+03,
            -1.47881538e+04, -1.68164681e+04, -1.09761700e+04,
            -7.58321814e-12],
           [ 0.00000000e+00,  4.21335690e+03,  6.45523729e+03,
             5.67664040e+03,  2.24188038e+03, -2.24188038e+03,
            -5.67664040e+03, -6.45523729e+03, -4.21335690e+03,
            -2.91092471e-12]] km>

In [197]:
smp = store.sat.orbit.sample(10)
sph = cartesian_to_spherical(smp.x,smp.y, smp.z)
sph[2]

<Longitude [0.        , 0.66451626, 1.38412353, 2.12465419, 2.81403748,
            3.46914783, 4.15853112, 4.89906177, 5.61866905, 6.28318531] rad>

In [241]:
@interact(ground_size=FloatSlider(min=0,max=30,value=10,continuous_update=False),
          n_sats = IntSlider(min=6,max=3600,value=18, continuous_update=False))
def groundshadow(ground_size, n_sats):
    # ground_size in degrees is the size of a tile on the surface
    # n_sats is the number of satelites on one repetion of an orbit
    # for heights > continuous coverage height, n_sats * grounds_size can be = 350
    # below continous coverage height, more n_sats are needed
    data = store.gp_groundtract.fig.data[1]
    ts = store.t_span-store.sat.orbit.epoch
    lats = interpolate.interp1d(ts.to(u.h), data['lat'])
    lons = interpolate.interp1d(ts.to(u.h), data['lon'])
    theta = np.linspace(2*np.pi,0)
    dt = store.sat.orbit.period.to(u.h)/n_sats
    t0 = ts[0].to(u.h)
    tend = ts[-1].to(u.h)
    for t in np.linspace(t0,tend,int(tend/dt)):
        store.gp_groundtract.fig.add_scattergeo(mode='text',fill='toself',fillcolor='blue', 
                      lat=lats(t)+ground_size/2*np.sin(theta),
                      lon=lons(t)+ground_size/2*np.cos(theta),
                                                opacity=0.1)
    store.gp_groundtract.update_geos()
    store.gp_groundtract.fig.show()

interactive(children=(FloatSlider(value=10.0, continuous_update=False, description='ground_size', max=30.0), I…

In [34]:
a = 7000 * u.km
ecc = 0 * u.one
inc = 10 * u.deg
raan = 0 * u.deg
argp = 0 * u.deg
nu = 0 * u.deg
orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
print("Original period: ", orb.period.to(u.h))
sat = EarthSatellite(orb, None)
t_span = time_range(orb.epoch, periods=505, end=orb.epoch+10 * u.h)
gp = GroundtrackPlotter()
gp.update_layout(title="Groundtrack")

# Plot previously defined EarthSatellite object
gp.plot(
    sat,
    t_span,
    label="Sat",
    color = "red",
    marker={"size": 10, "symbol": "triangle-right", "line": {"width": 1, "color": "black"}},
)

Original period:  1.6190323993572264 h


In [ ]:

    np.savetxt(f'cache/{h:.2f}_{i:.2f}_{duration}_{max_day_bn_rep}_{ground_size:.2f}_{n_sats}.csv',
               np.vstack([data['lat'],data['lon']]).T, delimiter=',')
    gp.update_geos(projection_type="orthographic")